## fine tuning using data from quora questions

In [1]:
import torch
from torch.utils.data import DataLoader
from sentence_transformers import losses, util
from sentence_transformers import LoggingHandler, SentenceTransformer, evaluation
from sentence_transformers.readers import InputExample
import logging
from datetime import datetime
import csv
import os
from zipfile import ZipFile
import random

In [2]:
#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)
#### /print debug information to stdout

In [3]:
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
# torch_device = 'cpu'

model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1',
                           device=torch_device)
num_epochs = 1#10
train_batch_size = 16

2022-11-11 13:07:59 - Load pretrained SentenceTransformer: multi-qa-MiniLM-L6-cos-v1


In [4]:
#As distance metric, we use cosine distance (cosine_distance = 1-cosine_similarity)
distance_metric = losses.SiameseDistanceMetric.COSINE_DISTANCE

#Negative pairs should have a distance of at least 0.5
margin = 0.5

dataset_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                            'data/quora-IR-dataset/')

model_save_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                               'data/output/training_OnlineConstrativeLoss-')\
+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")


os.makedirs(model_save_path, exist_ok=True)

In [5]:
# Check if the dataset exists. If not, download and extract
if not os.path.exists(dataset_path):
    logger.info("Dataset not found. Download")
    zip_save_path = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), 
                            'data/quora-IR-dataset/quora-IR-dataset.zip')
    util.http_get(url='https://sbert.net/datasets/quora-IR-dataset.zip', path=zip_save_path)
    with ZipFile(zip_save_path, 'r') as zip:
        zip.extractall(dataset_path)

In [6]:
######### Read train data  ##########
# Read train data
train_samples = []
with open(os.path.join(dataset_path, "classification/train_pairs.tsv"), encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:       
        sample = InputExample(texts=[row['question1'], row['question2']], label=int(row['is_duplicate']))
        train_samples.append(sample)



In [7]:
print(type(row['question1']), row['question2'])

<class 'str'> Why do people buy the iPhone?


In [8]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.OnlineContrastiveLoss(model=model, distance_metric=distance_metric, margin=margin)

In [9]:
################### Development  Evaluators ##################
# We add 3 evaluators, that evaluate the model on Duplicate Questions pair classification,
# Duplicate Questions Mining, and Duplicate Questions Information Retrieval
evaluators = []

###### Classification ######
# Given (quesiton1, question2), is this a duplicate or not?
# The evaluator will compute the embeddings for both questions and then compute
# a cosine similarity. If the similarity is above a threshold, we have a duplicate.
dev_sentences1 = []
dev_sentences2 = []
dev_labels = []
with open(os.path.join(dataset_path, "classification/dev_pairs.tsv"), encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        dev_sentences1.append(row['question1'])
        dev_sentences2.append(row['question2'])
        dev_labels.append(int(row['is_duplicate']))


binary_acc_evaluator = evaluation.BinaryClassificationEvaluator(dev_sentences1, 
                                                                dev_sentences2, 
                                                                dev_labels)
evaluators.append(binary_acc_evaluator)

In [30]:
###### Duplicate Questions Mining ######
# Given a large corpus of questions, identify all duplicates in that corpus.

# For faster processing, we limit the development corpus to only 10,000 sentences.
# max_dev_samples = 10000
max_dev_samples = 1000
dev_sentences = {}
dev_duplicates = []
with open(os.path.join(dataset_path, "duplicate-mining/dev_corpus.tsv"), encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        dev_sentences[row['qid']] = row['question']

        if len(dev_sentences) >= max_dev_samples:
            break

with open(os.path.join(dataset_path, "duplicate-mining/dev_duplicates.tsv"), encoding='utf8') as fIn:
    reader = csv.DictReader(fIn, delimiter='\t', quoting=csv.QUOTE_NONE)
    for row in reader:
        if row['qid1'] in dev_sentences and row['qid2'] in dev_sentences:
            dev_duplicates.append([row['qid1'], row['qid2']])

In [34]:
# The ParaphraseMiningEvaluator computes the cosine similarity between all sentences and
# extracts a list with the pairs that have the highest similarity. Given the duplicate
# information in dev_duplicates, it then computes and F1 score how well our duplicate mining worked
paraphrase_mining_evaluator = evaluation.ParaphraseMiningEvaluator(dev_sentences, 
                                                                   dev_duplicates, name='dev')
evaluators.append(paraphrase_mining_evaluator)

In [40]:
###### Duplicate Questions Information Retrieval ######
# Given a question and a large corpus of thousands questions, find the most relevant (i.e. duplicate) question
# in that corpus.

# For faster processing, we limit the development corpus to only 10,000 sentences.
# max_corpus_size = 100000
max_corpus_size = 100000

ir_queries = {}             #Our queries (qid => question)
ir_needed_qids = set()      #QIDs we need in the corpus
ir_corpus = {}              #Our corpus (qid => question)
ir_relevant_docs = {}       #Mapping of relevant documents for a given query (qid => set([relevant_question_ids])

with open(os.path.join(dataset_path, 'information-retrieval/dev-queries.tsv'), encoding='utf8') as fIn:
    next(fIn) #Skip header
    for line in fIn:
        qid, query, duplicate_ids = line.strip().split('\t')
        duplicate_ids = duplicate_ids.split(',')
        ir_queries[qid] = query
        ir_relevant_docs[qid] = set(duplicate_ids)

        for qid in duplicate_ids:
            ir_needed_qids.add(qid)

In [44]:
ir_corpus

{}

In [49]:
# First get all needed relevant documents (i.e., we must ensure, that the relevant questions are actually in the corpus
distraction_questions = {}
with open(os.path.join(dataset_path, 'information-retrieval/corpus.tsv'), encoding='utf8') as fIn:
    next(fIn) #Skip header
    for line in fIn:
        qid, question = line.strip().split('\t')

        if qid in ir_needed_qids:
            ir_corpus[qid] = question
        else:
            distraction_questions[qid] = question

# Now, also add some irrelevant questions to fill our corpus
other_qid_list = list(distraction_questions.keys())
random.shuffle(other_qid_list)

for qid in other_qid_list[0:max(0, max_corpus_size-len(ir_corpus))]:
    ir_corpus[qid] = distraction_questions[qid]

In [51]:
ir_corpus

{'15': 'How can I be a good geologist?',
 '102': 'How is career launcher online program for RBI Grade B?',
 '104': 'How can you play a Blu Ray DVD on a regular DVD player?',
 '116': 'What are some of the best rap songs?',
 '125': "How is the new Harry Potter book 'Harry Potter and the Cursed Child'?",
 '126': 'How bad is the new book by J.K Rowling?',
 '130': 'Where do I find a U.S. family office database?',
 '208': 'Which are some of the best fiction novels?',
 '216': 'What is the difference between love and pity?',
 '245': 'What mineral holds the highest electrical charge?',
 '268': 'Why am I attracted to older men?',
 '273': 'Does it matter whether humans are selfish or evil?',
 '288': 'Does a black hole have a finite mass?',
 '449': 'Will there be another billion dollar lottery Jackpot?',
 '539': 'If I am in prison, what rights would I have?',
 '596': 'On what online platforms can I post ads for beer money opportunity?',
 '620': 'Can I use Jio SIM in iPhone 5s?',
 '738': 'Have you 

In [14]:
#Given queries, a corpus and a mapping with relevant documents, the InformationRetrievalEvaluator computes different IR
# metrices. For our use case MRR@k and Accuracy@k are relevant.
ir_evaluator = evaluation.InformationRetrievalEvaluator(ir_queries, ir_corpus, ir_relevant_docs)

evaluators.append(ir_evaluator)

In [15]:
# Create a SequentialEvaluator. This SequentialEvaluator runs all three evaluators in a sequential order.
# We optimize the model with respect to the score from the last evaluator (scores[-1])
seq_evaluator = evaluation.SequentialEvaluator(evaluators, 
                                               main_score_function=lambda scores: scores[-1])


logger.info("Evaluate model without training")
with torch.no_grad():
    seq_evaluator(model, epoch=0, steps=0, output_path=model_save_path)

2022-11-09 19:58:58 - Evaluate model without training
2022-11-09 19:58:58 - Binary Accuracy Evaluation of the model on  dataset in epoch 0 after 0 steps:
2022-11-09 19:59:38 - Accuracy with Cosine-Similarity:           78.65	(Threshold: 0.8316)
2022-11-09 19:59:38 - F1 with Cosine-Similarity:                 72.70	(Threshold: 0.7516)
2022-11-09 19:59:38 - Precision with Cosine-Similarity:          62.64
2022-11-09 19:59:38 - Recall with Cosine-Similarity:             86.61
2022-11-09 19:59:38 - Average Precision with Cosine-Similarity:  75.95

2022-11-09 19:59:38 - Accuracy with Manhattan-Distance:           78.67	(Threshold: 9.0634)
2022-11-09 19:59:38 - F1 with Manhattan-Distance:                 72.66	(Threshold: 11.1130)
2022-11-09 19:59:38 - Precision with Manhattan-Distance:          61.97
2022-11-09 19:59:38 - Recall with Manhattan-Distance:             87.82
2022-11-09 19:59:38 - Average Precision with Manhattan-Distance:  75.94

2022-11-09 19:59:38 - Accuracy with Euclidean-Di

In [16]:
# Train the model
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=seq_evaluator,
          epochs=num_epochs,
#           warmup_steps=1000,
          warmup_steps=500,
          output_path=model_save_path
         )

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/17395 [00:00<?, ?it/s]

2022-11-09 20:33:25 - Binary Accuracy Evaluation of the model on  dataset after epoch 0:
2022-11-09 20:34:04 - Accuracy with Cosine-Similarity:           84.07	(Threshold: 0.8343)
2022-11-09 20:34:04 - F1 with Cosine-Similarity:                 78.60	(Threshold: 0.7941)
2022-11-09 20:34:04 - Precision with Cosine-Similarity:          70.91
2022-11-09 20:34:04 - Recall with Cosine-Similarity:             88.17
2022-11-09 20:34:04 - Average Precision with Cosine-Similarity:  83.50

2022-11-09 20:34:04 - Accuracy with Manhattan-Distance:           84.01	(Threshold: 8.9001)
2022-11-09 20:34:04 - F1 with Manhattan-Distance:                 78.57	(Threshold: 9.9410)
2022-11-09 20:34:04 - Precision with Manhattan-Distance:          71.06
2022-11-09 20:34:04 - Recall with Manhattan-Distance:             87.85
2022-11-09 20:34:04 - Average Precision with Manhattan-Distance:  83.45

2022-11-09 20:34:04 - Accuracy with Euclidean-Distance:           84.07	(Threshold: 0.5757)
2022-11-09 20:34:04 - 